In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from preprop import *

In [2]:
df = pd.read_csv('samples/pet.label.csv')
df.shape

(17458, 10)

In [3]:
df.head()

,보호자명,환자명,품종코드,생일,성별코드,검사일,병원,주진단코드,진단코드,SE
0,김소야*7,코코,91.0,2001-05-03 00:00:00,MN,2018-07-04 10:51:01.657,D,2078,"2078,만성신질환, 신장결석, 방광염","S) 식욕부진, 기력저하, 눈 O) - 2일동안 식욕이 거의 없어서 보호자분..."
1,이지은*6,힘찬이,119.0,2007-06-04 00:00:00,MN,2018-04-26 11:33:10.467,D,2244,"2244,담낭점액종","S) 다음다뇨, 식욕부진 O) - 최근들어 다음다뇨 증상과 2~3일전부터는 ..."
2,홍성미,비쥬,499.0,2004-01-01 00:00:00,FS,2018-07-20 14:48:07.967,D,2082,"2082,만성신질환, 부비동염","[오전 통화 by 관중] - 일주일동안 밥안먹고, 활력저하됨 - 보호자님 복부촉..."
3,김래경,세순이,107.0,2001-08-01 00:00:00,FS,2017-06-16 14:56:11.267,D,2087,"2087,단백뇨, 쿠싱, 빈혈",1. 쿠싱약 안먹고 와서 쿠싱검사는 28일에 하기로 함 2. 흉방에서 CVC 허탈...
4,김지현*6,효,125.0,2009-11-05 00:00:00,MN,2018-02-28 10:26:20.280,D,2081,"2081,방광결석","S) 건강검진 진행함 O) 1. 혈액검사 - 간수치 상승(ALP, ALT ..."


In [4]:
print(df.shape)
df.dropna(subset=['SE'], inplace=True)
print(df.shape)
df.dropna(subset=['주진단코드'], inplace=True)
print(df.shape)

(17458, 10)
(15974, 10)
(14204, 10)


In [5]:
df['len'] = df.SE.apply(lambda x : len(str(x)))
print(df.shape)
df = df[df['len'] > 50]
df = df[df['len'] < 5000]
print(df.shape)

(14204, 11)
(13577, 11)


In [7]:
df.SE = df.SE.apply(lambda x : preprocess(str(x)))

In [9]:
df['주진단코드'] = df['주진단코드'].apply(lambda x : label_regex(x))

In [10]:
print(df.shape)
df = df[df['주진단코드']!='normal']
print(df.shape)

(13577, 11)
(12224, 11)


In [11]:
df['주진단코드'].value_counts()[0:40]

중성화     580
2046    564
2043    534
2001    487
2087    479
2082    479
2101    377
2071    324
2170    313
2092    251
2133    208
2116    200
2275    199
2095    174
건강검진    156
2126    137
2185    136
2048    127
2091    121
2274    120
2002    120
2031    114
2244    112
2022    107
2181    104
2259    103
2273    100
2255     93
2236     92
2245     86
2120     83
2267     82
2020     80
2283     79
2228     79
2289     76
2230     76
2049     75
2081     73
2084     71
Name: 주진단코드, dtype: int64

In [12]:
label_dict = df['주진단코드'].value_counts()[0:40].to_dict()
def make_label_id(label) :
    if not label in label_dict.keys() :
        label = np.nan
    return label

In [13]:
df = df.reset_index()

In [14]:
diag = pd.read_csv('samples/diag.csv')
diag['질병코드'] = diag['질병코드'].apply(lambda x : str(x).split('.')[0])
diag.head()

,Unnamed: 0,분류,질병코드,질병명(한글)
0,0,순환기 질환,2001,판막증 (의심 포함한 심장 잡음 + 심부전 증후 자)
1,1,순환기 질환,2002,판막증 (심장 마비 증후 +)
2,2,순환기 질환,2003,병태 불명의 심비대 / 마음 확대
3,3,순환기 질환,2004,심근증
4,4,순환기 질환,2005,부정맥


In [15]:
df['질병명'] = ''
for i in range(len(df)) :
#     print(i)
#     diag = df_diag[df.loc[i, '진단코드'] == df_diag['질병코드']]['질병명(한글)']
#     print(df.loc[i, '질병명'])
#     print(df.loc[i, '주진단코드'])
    if str(df.loc[i, '주진단코드']).strip() in diag['질병코드'].to_list() :
#         print(df.loc[i, '주진단코드'])
#         print(diag[df.loc[i, '주진단코드'] == diag['질병코드']]['질병명(한글)'].values[0])
        df.loc[i, '질병명'] = diag[df.loc[i, '주진단코드'] == diag['질병코드']]['질병명(한글)'].values[0]
    if (str(df.loc[i, '주진단코드']).strip() == '건강검진') :
        df.loc[i, '질병명'] = '건강검진'
    if (str(df.loc[i, '주진단코드']).strip() == '중성화') :
        df.loc[i, '질병명'] = '중성화'        
#         df.loc[i, '분류'] = df_diag[df.loc[i, '진단코드'] == df_diag['질병코드']]['분류'].values[0]

In [16]:
df = df[['질병명', '주진단코드', 'SE']]

In [17]:
df['질병명'].value_counts()

                                  852
중성화                               580
 소화관 이물 / 섭취                      564
 위염 / 위장염 / 장염                    534
 판막증 (의심 포함한 심장 잡음 + 심부전 증후 자)    487
                                 ... 
 流涙症(눈물 화상 / 누관 폐쇄 포함)              1
 노콧 증                               1
 반대로 재채기 증후군 · RSS                  1
 귀의 질환                              1
 대동맥 협착증 · AS                       1
Name: 질병명, Length: 270, dtype: int64

In [18]:
diag_dict ={}
for i in range(len(df)) :
    diag_dict[df.loc[i, '주진단코드']] = df.loc[i, '질병명']

In [19]:
df['진단코드'] = ''
for i in range(len(df)) :
#     print(str(df.loc[i, '주진단코드']).strip())
    if str(df.loc[i, '주진단코드']).strip() == '2092' or str(df.loc[i, '주진단코드']).strip() == '2091':
        df.loc[i, '진단코드'] = '방광질환'
    if str(df.loc[i, '주진단코드']).strip() == '2031' or str(df.loc[i, '주진단코드']).strip() == '2022':
        df.loc[i, '진단코드'] = '폐질환'      
    if str(df.loc[i, '주진단코드']).strip() == '2001' or str(df.loc[i, '주진단코드']).strip() == '2002':
        df.loc[i, '진단코드'] = '심장질환'    
    if str(df.loc[i, '주진단코드']).strip() == '2048' or str(df.loc[i, '주진단코드']).strip() == '2043':
        df.loc[i, '진단코드'] = '위장염'   
    if str(df.loc[i, '주진단코드']).strip() == '2084' or str(df.loc[i, '주진단코드']).strip() == '2081':
        df.loc[i, '진단코드'] = '간질환'           
    if str(df.loc[i, '질병명']).strip() == '중성화':
        df.loc[i, '진단코드'] = '중성화'       
    if str(df.loc[i, '질병명']).strip() == '건강검진': 
        df.loc[i, '진단코드'] = '건강검진'      
    if str(df.loc[i, '주진단코드']).strip() == '2046': 
        df.loc[i, '진단코드'] = '소화관 이물' 
    if str(df.loc[i, '주진단코드']).strip() == '2082': 
        df.loc[i, '진단코드'] = '췌장염'  
    if str(df.loc[i, '주진단코드']).strip() == '2087': 
        df.loc[i, '진단코드'] = '신장질환'  
    if str(df.loc[i, '주진단코드']).strip() == '2101': 
        df.loc[i, '진단코드'] = '자궁 축농증'    
    if str(df.loc[i, '주진단코드']).strip() == '2071': 
        df.loc[i, '진단코드'] = '구토설사혈변'    
    if str(df.loc[i, '주진단코드']).strip() == '2170': 
        df.loc[i, '진단코드'] = '치주염'    
    if str(df.loc[i, '주진단코드']).strip() == '2116': 
        df.loc[i, '진단코드'] = '유선종양'    
    if str(df.loc[i, '주진단코드']).strip() == '2095': 
        df.loc[i, '진단코드'] = '고양이하부요로질환'      
    if str(df.loc[i, '주진단코드']).strip() == '2126': 
        df.loc[i, '진단코드'] = '경련 발작'    
    if str(df.loc[i, '주진단코드']).strip() == '2185': 
        df.loc[i, '진단코드'] = '슬개골탈구'        
    if str(df.loc[i, '주진단코드']).strip() == '2274': 
        df.loc[i, '진단코드'] = '교상'      
    if str(df.loc[i, '주진단코드']).strip() == '2244': 
        df.loc[i, '진단코드'] = '당뇨병'              

In [20]:
df['진단코드'] = df['진단코드'].apply(lambda x : np.nan if str(x) == '' else x)

In [21]:
print(df.shape)
df.dropna(subset=['진단코드'], inplace=True)
print(df.shape)

(12224, 4)
(6156, 4)


In [22]:
df['진단코드'].value_counts()

위장염          661
심장질환         607
중성화          580
소화관 이물       564
신장질환         479
췌장염          479
자궁 축농증       377
방광질환         372
구토설사혈변       324
치주염          313
폐질환          221
유선종양         200
고양이하부요로질환    174
건강검진         156
간질환          144
경련 발작        137
슬개골탈구        136
교상           120
당뇨병          112
Name: 진단코드, dtype: int64

In [23]:
import pandas as pd
import numpy as np
import os
import re
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [24]:
encoder = LabelEncoder()
encoder.fit(df['진단코드'])
df['label_id'] = encoder.transform(df['진단코드'])

In [25]:
df.head()

,질병명,주진단코드,SE,진단코드,label_id
1,당뇨병,2244,s 다음다뇨 식욕부진 최근들어 다음다뇨 증상과 month 전부터는 식욕도 약간 떨어...,당뇨병,6
2,췌장염,2082,오전 통화 by 관중 일주일동안 밥안먹고 활력저하됨 보호자님 복부촉진식 통증은 확...,췌장염,16
3,만성 신장 질환 (신부전 포함),2087,쿠싱약 안먹고 와서 쿠싱검사는 time 에 하기로 함 흉방에서 cvc 허탈 심해서...,신장질환,10
4,간 효소 상승 (원인 미정),2081,s 건강검진 진행함 혈액검사 간수치 상승 alp alt 모두 상승함 영상검사 신장의...,간질환,0
5,당뇨병,2244,money 선결제 하셨습니다. 송이 식욕부진 구토 month 전부터 식욕부진 구토...,당뇨병,6


In [ ]:
train, test = train_test_split(df,test_size=0.1, random_state=100 )
train.to_csv('files/train1.csv', index=None)
test.to_csv('files/test1.csv', index=None)
print(train.shape)
print(test.shape)

In [ ]:
train = pd.read_csv('files/train1.csv')
test = pd.read_csv('files/test1.csv')
df = pd.concat([train, test])
df.shape

In [26]:
label_dict1 = df['진단코드'].value_counts()[0:10].to_dict()
def make_label_id1(label) :
    if not label in label_dict1.keys() :
        label = np.nan
    return label

In [27]:
# df['label_id'] = ''
df['진단코드'] = df['진단코드'].apply(lambda x : make_label_id1(x))
print(df.shape)
df.dropna(subset=['진단코드'], inplace=True)
print(df.shape)

(6156, 5)
(4756, 5)


In [ ]:
encoder = LabelEncoder()
encoder.fit(df['진단코드'])
df['label_id'] = encoder.transform(df['진단코드'])

In [ ]:
train, test = train_test_split(df,test_size=0.1, random_state=100 )
train.to_csv('files/train2.csv', index=None)
test.to_csv('files/test2.csv', index=None)
print(train.shape)
print(test.shape)

In [ ]:
#########